# Ejercicio 4 - Clasificación No-Supervisada

Esta actividad propone trabajar con clasificación no-supervisada o clustering sobre un dataset de información de clientes bancarios a los que se los contacta para ofrecer un nuevo producto. Cada cliente se describe con diferentes atributos.

## Contexto: Análisis de éxito en campaña de marketing

Fuente: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

El foco está en realizar un análisis de agrupamiento o segmentación (clustering)de un *dataset* de información de un resultados de personas contactadas por una campaña de marketing y que compraron la oferta (atributo "OK"), con cerca de 41.200 registros de personas contactadas.

Este conjunto de datos (abierto para este tipo de usos instruccionales), consiste en 20 atributos originales y 1 clase de etiquetas (totalizando 21 columnas) y corresponde a los datos de una campaña telefónica a diversos clientes en Portugal, ofreciéndoles la compra de un producto bancario. En varios casos, un cliente fue contactado varias veces antes de aceptar el el depósito a plazo ofrecido por la campaña (OK = 1).

Algunos de los atributos relevantes son (combinando atributos categóricos, con numéricos):
* **Datos personales**: Edad, Ocupación, Estado Civil, Nivel de Educación.
* **Datos financieros**: Su casa tiene crédito hipotecario, default: si el crédito ha caído en quiebra; tiene un crédito de consumo.
* **Datos de contactos de la campaña actual**: Tipo de Comunicación (celular o teléfono fijo); Mes del último contacto; Día de la semana del contacto; duración de la llamada (segundos); Contacto: N° de contactos durante la campaña; DíasAtrás: días transcurridos desde último contacto; Resultado: resultado de la última llamada (falló, no-existe, éxito)
* **Datos socioeconómicos**: EmpTasaVar: tasa de variación de empleabilidad; IPC: índice de precios consumidor mensual; ICC: índice de confianza consumidor mensual; Euribor3m: tasa euribor de 3 meses indicador diario; NumEmpleados: cantidad de gente empleada, en indicador trimestral.

Esta adaptación en particular, por el equipo de R:Solver (RSolver.com), enfrenta diferentes objetivos de aprendizaje dentro de los cursos de Big Data y Machine Learning.


**INSTRUCCIONES**
Todos los alumnos deben contestar las preguntas que se indican en las secciones de "Preguntas", más adelante. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases. Deben entregar un informe simple (de 1 ó 2 páginas) en formato PDF solamente, en el que se incluyen los nombres de los integrantes ordenados alfabéticamente por apellido dentro de la primera página.

La plantilla de informe está en http://dcc.rsolver.com/dcc/docs/InformeActividad.docx

El informe en formato PDF debe ser subido por sólo uno de los integrantes a la siguiente URL:

http://aiker.rsolver.com/aiker/DocUpload.aspx (*)

(*) Sólo y si hay problemas en la carga, enviar el PDF a rsandova@ing.puc.cl y cc: ayudante@rsolver.com


## Actividad Complementaria: Instalar librerías de modelos de clasificación y herramientas

In [ ]:
install.packages('e1071')
install.packages('caret')
install.packages('caTools')
install.packages('corrplot')
install.packages('ggplot2')
install.packages('fpc')
install.packages('cluster')
install.packages('dbscan')
install.packages('Hmisc')

## Actividad Complementaria: Carga de los datos y preprocesamiento

La siguiente celda de código carga los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la disponibilidad de los datos. Nótese que esta versión de los datos está adaptada a trabajar mayormente con valores numéricos.

In [ ]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://www.rsolver.com/dcc/docs/bank-additional-full-numbers.csv"

# Se declaran los nombres de las columnas
columnas = c("Edad","Ocupación","EstadoCivil","Educación","Default","Hipotecario","Consumo","Contacto","Mes","Día",
             "Duración","NumContactos","DíasAtrás","Previo","ResultadoPrevio", "EmpTasaVar", "IPC", "ICC", "Euribor3m", "NumEmpleados", "OK")

# Se cargan datos principales a una estructura (marketing.data), asignando nombres de atributos a las columnas
marketing.data <- read.table(file = theUrlMain, header = TRUE, sep = ";", col.names = columnas, na.strings = c("unknown","NA"))

# Se eliminan los registros que tienen algún NA (unknown)
marketing.data <- na.omit(marketing.data) # Sólo quedan poco más de 30.000 filas (de las 41.000 originales)

summary(marketing.data)


## Complemento: visualización de la correlación entre algunas de las variables

En el siguiente segmento de código se usa la librería *corrplot* para visualizar correlaciones sólo entre los atributos numéricos del dataset, que puede ayudar a determinar cuál es de ellos serían los ideales de usar en la ejecución de HDBSCAN. En la práctica, pares de atributos con una alta correlación no aportan diversidad en un análisis de conjuntos, pero atributos con baja correlación pueden llevar a una diversidad en su descripción.

In [ ]:
library("corrplot")

# referencia: https://cran.r-project.org/web/packages/dbscan/vignettes/hdbscan.html
# En este caso sólo se correlacionan las variables numéricas
cor <- cor(marketing.data[c(1, 5:7, 11:14, 16:20)])
corrplot(cor, method = "pie", type ="upper")

## Ejercicio: Realizar una buena segmentación e interpretar resultados

El objetivo de ejecutar una segmentación o clustering es determinar automáticamente los conjuntos que contienen individuos con características similares. En el contexto del caso de análisis (clientes de campaña marketing de banco) se vuelve interesante producir una segmentación con pocos clústers, para -eventualmente- facilitar el análisis de cada clúster en forma individual. Por ello, se define que el resultado esperado es de entre 5 y 20 clústers, incluyendo el grupo de los "ruidosos" (*noise*) y que los conjuntos (salvo el de los ruidosos) sean relativamente balanceados, es decir que idealmente no haya un súper clúster ni tampoco micro-clústers.

Entonces, a continuación se tienen las líneas de código que generan una primera segmentación, la cual hay que refinar, buscando dos objetivos:

**OBJETIVO 1 (4 puntos)**

Buscar una configuración del proceso que encuentre clústers que mejor segmentan los datos.

Esto plantea ir probando combinaciones de:

-     Cuáles tres atributos del total de atributos son los que logran producir los mejores(*) conjuntos finales. Se propone correr el modelo con tríos de atributos para ver cuál logra mejores resultados. (Pueden usar como referencia la información que entrega el plot de correlación anterior, para acotar las posibles combinaciones)
-     El valor del parámetro minPts del algoritmo HDBSCAN. Se propone probar valores: 100, 200, 400, 600.

(*) Se define que los mejores conjuntos finales son aquellos en que:

-    Los grupos o clusters son relativamente balanceados en cantidad de elementos (ningún clúster tiene más del 30% del total de datos).
-    La cantidad de puntos de ruido (*noise*) es menos del 20% del total de puntos (es decir poco más de 6.000).

**OBJETIVO 2 (2 puntos)**

Describir las características de los clusters en relación a los valores de sus atributos más comunes o promediados. La idea es poder obtener una interpretación más clara de qué caracteriza a cada uno de los clusters.

Esta interpretación puede ser una declaración verbal de la siguiente forma:

*      "Los clusters resultantes evidencian un balance, donde sólo hay un único cluster dominante, aunque no tiene más del 30% de los elementos..."

*      "El clúster 2 y 5 tienen clientes que se asemejan por el atributo XYZ, dado que el valor promedio y el rango son similares, mientras que los otros clúster difieren en..."

*      "Los primeros 5 clústers agrupan a los clientes que dicen OK a la campaña, con un promedio de edad de..."

Se espera que los alumnos puedan aprovechar éstas (y/u otras) herramientas gráficas para realizar una interpretación general del resultado del proceso de clustering.

Una importante recomendación es utilizar herramientas de cálculo de rangos y promedios de los atributos de los clientes en cada cluster, según el trío de atributos elegido.

Por ej, si se usó el atributo Edad, entonces determinar que los clientes de un clúster tienen edades en rango de 32,5 a 50, con un promedio de 42, versus otro clúster que tiene clientes en rango de edad mayor o menor, es una forma útil de determinar lo que diferencia a cada clúster de los otros.

Otro ej: si se usa el ICC (índice de confianza de consumidor) ver que un clúster asocia clientes contactados con ICC en promedio mayor que los otros clústers es una forma de describir con más claridad la diferencia entre los grupos.


In [ ]:
library (dbscan)

# Ejercicio 1: determinar los atributos (elegir 3, además de OK) que mejor agrupen los datos de los clientes.
# Como ejemplo se usa la Edad, Consumo, IPC. IMPORTANTE: Según las columnas elegidas, puede ser necesario cambiar parte del código de la visualización (OBJETIVO 2)
# Además elegir el valor de minPts que logre la cantidad esperada de clústers.
dataForClustering <- data.frame(marketing.data$Edad, marketing.data$Consumo, marketing.data$IPC, marketing.data$OK)

# Ejecución de HDBSCAN (en este ejercicio toma menos de 30 seg)
# Probar minPts = 100, 200, 400, 600 u otros valores (Ojo: Podría ocurrir que un minPts muy grande genere error por falta de memoria RAM)
clustering <- hdbscan(dataForClustering, minPts = 100)

# Viusalización del resultado del clustering en forma de texto descriptivo
# Nota: el dataset "clustering" tiene columnas: cluster, minPts, cluster_scores, membership_prob, outliers_scores, hc
clustering
summary(clustering)

In [ ]:
# Ejercicio 2: Visualización y Análisis
# Se presentan alternativas básicas de visualización, esperando que los alumnos las utilicen (y/u otras)
# para concluir una explicación razonable y adecuada del resultado.

# 1) Usar el PLOT de Jerarquía para mostrar el Árbol HDBSCAN
# Referencia: https://cran.r-project.org/web/packages/dbscan/vignettes/hdbscan.html
# Como explicación: los "objetos" en la parte inferor son los clústers
plot(clustering, col=clustering$cluster+1, pch=30, show_flat = T, scale=10)
points(dataForClustering, pch=20)

# 2) Cluster Plot por los mejores 2 principal components
library(cluster)
clusplot(dataForClustering, clustering$cluster, color=TRUE, shade=TRUE, labels=2, lines=0)

# 3) Centroid Plot
library(fpc)
plotcluster(dataForClustering, clustering$cluster)



In [ ]:
# 4) Análisis de los valores detallados por clúster por un atributo.
# Nótese que se puede/debe cambiar el atributo o columna a analizar
table(clustering$cluster, marketing.data$OK)
